問題

## (1) 物体検出の分野にはどういった手法が存在したか。

Object detectionではクラスとバウンディングボックスを検出する構造と物体候補領域の推定の構造を持つ

(Related Work)Widely used object proposal methods include those based on grouping super-pixels (eg, Selective Search [4], CPMC [22], MCG [23]) and those based on sliding windows (eg, objectness in windows [24], EdgeBoxes [6])

より、物体候補領域の推定の構造にslideing windowやSelective Searchが用いられていた。
sliding window は単純に画像にwindowを網羅的に当てはめて検索する、SelectiveSearchはセグメンテーションのように画像の中の物体の特徴を階層的に抽出して結合する方法で、複数のスケールのウィンドウを調べて、テクスチャ、色、または強度を共有する隣接ピクセルを探し物体を識別する。

クラスとバウンディングボックスを検出する構造では
The R-CNN method [5] trains CNNs end-to-end to classify the proposal regions into object categories or background.
より、R-CNN法[5]は、提案領域をオブジェクトカテゴリまたは背景にクラス分類する。

OverFeat method [9], a fully-connected layer is trained to predict the box coordinates for the localization task that assumes a single
object.より物体の座標や境界をOverFeat法で予測していた。

## (2) Fasterとあるが、どういった仕組みで高速化したのか

(Abstract)　Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, exposing region proposal computation as a bottleneck.よりSPPnet [1]やFast R-CNN [2]では実行時間が減少するもResionProposalのSelectiveSearchの部分がボトルネックになっていた。

(Abstract) we introduce a Region Proposal Network (RPN) that shares full-image convolutional features with the detection network, thus enabling nearly cost-free region proposals.(3 Faster R-CNN)RPN module tells the Fast R-CNN module where to look. とあるので、FasterR-CNNではFastR-CNNのend-to-endな検出構造に加えて領域推定の部分もRegion Proposal NetworkというCNNを導入し、Fast R-CNNに物体がありそうな領域を提案することで高速化し、全体としてend-to-endで訓練できるようになった。


## (3)One-Stageの手法とTwo-Stageの手法はどう違うのか。

One-stageは1つのCNN内で
バウンディングボックスをクラス分類を行う方法で
Two-stageは物体領域候補を予測して次の段階で座標の特定とクラス分類を行う。
この論文でのOne-stageはOverFeatを扱っている.OverFeatはpaperによるとスライディングウィンドウを畳み込み層内で効率的に行い、領域予測と分類を行う方法。

Though both methods use sliding windows, the region proposal task is only the first stage of Faster R-CNN—the downstream Fast R-CNN detector attends to the proposals to refine them.
どちらの方法もスライディングウィンドウを使用するが、RPNはFasterR-CNNの最初の段階にすぎず、下流のFastR-CNNディテクタで領域候補を改良するように手を加えるため精度がよくなる。

Table X compares the two-stage system and two variants of the one-stage system. Using the ZF model, the one-stage system has an mAP of 53.9%. This is lower than the two-stage system (58.7%) by 4.8%.
テーブルXから二段階の方が一段階のシステムよりmAPが4.8%いいことがわかる

## (4)RPNとは何か。

(3.1 Region Proposal Networks)A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.より、Region Proposal Network（RPN）は（任意のサイズの）画像を入力として受け取り、それぞれが物体らしさのスコアを持つ1組の長方形のオブジェクト予測を出力すネットワークのことであり、
Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers.から最終的な目標はFastR-CNNオブジェクト検出ネットワーク[2]とRPNが計算を共有することであるので、両方のネットワークが共通のたたみ込み層を共有すると仮定している。

## (5) RoIプーリングとは何か。

FastR-CNNの分野なのでその論文を読む
The RoI pooling layer uses max pooling to convert the
features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7),
where H and W are layer hyper-parameters that are independent of any particular RoI.
ＲｏＩプーリングレイヤは、マックスプーリングを使用して任意の有効なROIのfeature-mapをＨ×Ｗ（例えば７×７）の固定サイズのfeatur-mapに変換する。これによりR-CNNでは必要な入力前の画像のリサイズを行う必要がない.

## (6) Anchorのサイズはどうするのが適切か。

(3.1 Region Proposal Networks)To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. At each sliding-window location, we simultaneously predict multiple region proposals, where the number of maximum possible proposals for each location is denoted as k
領域提案を生成するために、最後の共有たたみ込みレイヤによって出力されたたたみ込み特徴マップ上に小さなネットワークをスライドさせる。各スライディングウィンドウ位置で複数の領域提案を予測し、各位置に対する最大可能提案boxの数はkで表される。

By default we use 3 scales and 3 aspect ratios, yielding k=9 anchors at each sliding position.論文によるとアンカー数kはデフォルトでスケールごとに3種類、アスペクト比ごとに3種類のk=9を採用している

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

dataset and metrics

We comprehensively evaluate our method on the PASCAL VOC 2007 detection benchmark

PASCAL VOC 2007というデータセットを使い、評価している

This dataset consists of about 5k trainval images and 5k test images over 20 object categories

このデータセットは5000のtrainと5000のtestイメージがあり、20クラスの物体カテゴリを有する

We primarily evaluate detection mean Average Precision (mAP), because this is the actual metric for object detection (rather than focusing on object proposal proxy metrics).
精度の評価はmAPで行う。これは実際の物体検出の指標値なため。
Table X compares the two-stage system and two variants of the one-stage system. Using the ZF model, the one-stage system has an mAP of 53.9%. This is lower than the two-stage system (58.7%) by 4.8%
One-stageのモデルのmAPが53.9%でFaster-RCNNは58.7%となり4.8%上回った



We also provide results on the PASCAL VOC 2012 benchmark for a few models.

いくつかのモデルではPASCAL VOC 2012のデータセットでも評価している

We present more results on the Microsoft COCO object detection dataset
マイクロソフトのCOCOというデータセットを使用している
 We experiment with the 80k images on the training set, 40k images on the validation set, and 20k images on the test-dev set.
80000個のtrain, 40000個のvalidation, 20000個のtest

In Table XI we first report the results of the Fast R-CNN system [2] using the implementation in this paper. Our Fast R-CNN baseline has 39.3% mAP@0.5 on the test-dev set, higher than that reported in [2].Next we evaluate our Faster R-CNN system. Using the COCO training set to train, Faster R-CNN has 42.1% mAP@0.5 and 21.5% mAP@[.5, .95] on the COCO test-dev set. This is 2.8% higher for mAP@0.5 and 2.2% higher for mAP@[.5, .95] than the Fast R-CNN counterpart under the same protocol (Table XI)
Fast R-CNNは原著論文で35.9のmAP,この論文の測定では39.7mAPであった。FasterR-CNNは
同じプロトコルの下でのFast R-CNN対応物よりも、mAP @ 0.5では2.8％、mAP @ [。5、.95]では2.2％高い結果であった

## (8) （アドバンス）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

YOLOでは

The recent Faster R-CNN replaces selective search with  a neural network to propose bounding boxes, similar to  Szegedy et al. [8] In our tests, their most accurate model  achieves 7 fps while a smaller, less accurate one runs at  18 fps. The VGG-16 version of Faster R-CNN is 10 mAP  higher but is also 6 times slower than YOLO. The ZeilerFergus Faster R-CNN is only 2.5 times slower than YOLO  but is also less accurate.


YOLOと同様に、選択的探索をニューラルネットワークに置き換えて境界ボックスを提案される比較対象として登場

Faster R-CNNの最も正確なモデルは7 fpsを達成しているが、より小さく、より正確でないモデルは18 fpsで動作
とあり、さらにFaster R-CNNのVGG-16バージョンは10 mAP高いがYOLOよりも6倍遅く、 ZeilerFergus Faster R-CNNはYOLOより2.5倍遅いだけでなく、正確さも劣ると言われている。